In [1]:
import pandas as pd
import numpy as np
import collections
from pandas import Series
import string
import warnings
warnings.filterwarnings('ignore')

In [2]:
campaigns = pd.read_csv('campaign_data.csv')
customers = pd.read_csv('customer_demographics.csv')
transactions = pd.read_csv('customer_transaction_data.csv')
coupons = pd.read_csv('coupon_item_mapping.csv')
items = pd.read_csv('item_data.csv')
train = pd.read_csv('train.csv')

In [3]:
campaigns.head()

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,7/9/2013,16/11/13
3,23,Y,8/10/2013,15/11/13
4,21,Y,16/09/13,18/10/13


In [4]:
coupons.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [5]:
transactions.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [6]:
campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   campaign_id    28 non-null     int64 
 1   campaign_type  28 non-null     object
 2   start_date     28 non-null     object
 3   end_date       28 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.0+ KB


In [7]:
coupons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92663 entries, 0 to 92662
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   coupon_id  92663 non-null  int64
 1   item_id    92663 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB


In [8]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324566 entries, 0 to 1324565
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1324566 non-null  object 
 1   customer_id      1324566 non-null  int64  
 2   item_id          1324566 non-null  int64  
 3   quantity         1324566 non-null  int64  
 4   selling_price    1324566 non-null  float64
 5   other_discount   1324566 non-null  float64
 6   coupon_discount  1324566 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 70.7+ MB


We want to explore if customers redeemed coupons in the same campaign multiple times. We look at campaign id 13 below and see that customer 1 redeemed 18 coupons, customer 2 19 coupons and so on

In [9]:
train[train['campaign_id']==13].groupby('customer_id').nunique()

,id,campaign_id,coupon_id,customer_id,redemption_status
customer_id,,,,,
1,18,1,18,1,1
3,19,1,19,1,1
5,26,1,26,1,1
7,20,1,20,1,1
8,22,1,22,1,2
...,...,...,...,...,...
1574,34,1,33,1,2
1578,17,1,17,1,1
1579,20,1,20,1,1


Further, we look at how many unique coupon_id's are associated with each campaign in the training set.

In [10]:
train.groupby('campaign_id')['coupon_id'].nunique()

campaign_id
1      11
2      16
3      34
4      12
5      11
6       1
7       8
8     208
9      18
10     14
11     13
12     15
13    207
26    181
27     27
28     28
29     33
30    178
Name: coupon_id, dtype: int64

Similarly, we look at how many unique campaigns are associated with each coupon and see that some coupons have been used for more than one campaign.

In [11]:
coupons_to_campaigns = train.groupby('coupon_id').agg(['unique'])['campaign_id']
print(coupons_to_campaigns.shape)
coupons_to_campaigns

(866, 1)


,unique
coupon_id,
1,[26]
2,[26]
3,[29]
4,[30]
5,[30]
...,...
1108,[13]
1110,"[8, 30]"
1112,"[13, 1]"


In [12]:
coupons_to_campaigns['length'] = coupons_to_campaigns['unique'].str.len()
coupons_to_campaigns[coupons_to_campaigns['length']>1].sort_values(by='length').head()

,unique,length
coupon_id,,
243,"[29, 26]",2
889,"[8, 30]",2
885,"[8, 13]",2
870,"[9, 26]",2
864,"[27, 3]",2


Let's convert the date columns to datetime object

In [13]:
transactions['date'] = pd.to_datetime(transactions['date'], dayfirst=True)

In [14]:
campaigns['start_date'] = pd.to_datetime(campaigns['start_date'], dayfirst=True)
campaigns['end_date'] = pd.to_datetime(campaigns['end_date'], dayfirst=True)

In [15]:
transactions.sort_values(by='date').head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.00
113,2012-01-02,464,20697,1,92.26,-21.37,-35.62
114,2012-01-02,464,20717,2,28.50,-27.78,0.00
115,2012-01-02,464,21008,1,35.26,-17.81,0.00
116,2012-01-02,464,22243,2,118.97,-22.80,0.00


We want to investigate if customers can use a coupon discount on the same item multiple times in a single transaction. From our analysis below, it turns out that 533 customers have done that.

In [16]:
transactions_redeemed = transactions[transactions['coupon_discount']!=0]
items_unique = []
for customer in transactions_redeemed['customer_id'].unique():
    df_customer = transactions_redeemed[transactions_redeemed['customer_id']==customer]
    for date in df_customer['date'].unique():
        df_date = df_customer[df_customer['date']==date]
        items_unique.append(df_date['item_id'].is_unique)
counter = collections.Counter(items_unique)
print(counter)

Counter({True: 8611, False: 533})


We want to group the data by customer, item and transaction date and sum up the column values.

In [17]:
transactions_by_customer = transactions.groupby(['customer_id', 'item_id', 'date']).sum()
transactions_by_customer.head()

quantity  selling_price  other_discount  \
customer_id item_id date                                                  
1           4315    2013-06-20         1         201.97             0.0   
            4577    2012-11-07         2         120.40             0.0   
            4796    2012-10-18         1         106.50             0.0   
            4953    2012-02-21         1         142.12             0.0   
                    2012-05-17         1         142.12             0.0   

                                coupon_discount  
customer_id item_id date                         
1           4315    2013-06-20              0.0  
            4577    2012-11-07              0.0  
            4796    2012-10-18              0.0  
            4953    2012-02-21              0.0  
                    2012-05-17              0.0

Next we merge this with our customer and item data

In [18]:
customers.set_index('customer_id', inplace=True)
items.set_index('item_id', inplace=True)

transactions_by_customer = transactions_by_customer.merge(customers, left_index=True, right_index=True, how='right')
transactions_by_customer = transactions_by_customer.merge(items, how='left',  left_index=True, right_index=True)
transactions_by_customer.head()

quantity  selling_price  other_discount  \
customer_id item_id date                                                  
1           4315    2013-06-20         1         201.97             0.0   
            4577    2012-11-07         2         120.40             0.0   
            4796    2012-10-18         1         106.50             0.0   
            4953    2012-02-21         1         142.12             0.0   
                    2012-05-17         1         142.12             0.0   

                                coupon_discount age_range marital_status  \
customer_id item_id date                                                   
1           4315    2013-06-20              0.0       70+        Married   
            4577    2012-11-07              0.0       70+        Married   
            4796    2012-10-18              0.0       70+        Married   
            4953    2012-02-21              0.0       70+        Married   
                    2012-05-17              0.0       70+        Married   

                                rented family_size no_of_children  \
customer_id item_id date                                            
1           4315    2013-06-20       0           2            NaN   
            4577    2012-11-07       0           2            NaN   
            4796    2012-10-18       0           2            NaN   
            4953    2012-02-21       0           2            NaN   
                    2012-05-17       0           2            NaN   

                                income_bracket  brand   brand_type  \
customer_id item_id date                                             
1           4315    2013-06-20               4   2902  Established   
            4577    2012-11-07               4    115  Established   
            4796    2012-10-18               4    278  Established   
            4953    2012-02-21               4   1061  Established   
                    2012-05-17               4   1061  Established   

                                      category  
customer_id item_id date                        
1           4315    2013-06-20  Pharmaceutical  
            4577    2012-11-07         Grocery  
            4796    2012-10-18         Grocery  
            4953    2012-02-21         Grocery  
                    2012-05-17         Grocery

Let's check for null values in our data

In [19]:
transactions_by_customer.isnull().sum()

quantity                0
selling_price           0
other_discount          0
coupon_discount         0
age_range               0
marital_status     329215
rented                  0
family_size             0
no_of_children     519855
income_bracket          0
brand                   0
brand_type              0
category                0
dtype: int64

Since they have a lot of null values, we will drop marital_status and no_of_children columns from out dataset later. Now we want to extract information from date into new columns

In [20]:
transactions_by_customer.reset_index(level=2, inplace=True)
transactions_by_customer.head()

transactions_by_customer['day'] = transactions_by_customer['date'].dt.day
transactions_by_customer['month'] = transactions_by_customer['date'].dt.month
transactions_by_customer['year'] = transactions_by_customer['date'].dt.year
transactions_by_customer['week'] = transactions_by_customer['date'].dt.week
transactions_by_customer['weekday'] = transactions_by_customer['date'].dt.weekday
transactions_by_customer.head()

date  quantity  selling_price  other_discount  \
customer_id item_id                                                       
1           4315    2013-06-20         1         201.97             0.0   
            4577    2012-11-07         2         120.40             0.0   
            4796    2012-10-18         1         106.50             0.0   
            4953    2012-02-21         1         142.12             0.0   
            4953    2012-05-17         1         142.12             0.0   

                     coupon_discount age_range marital_status  rented  \
customer_id item_id                                                     
1           4315                 0.0       70+        Married       0   
            4577                 0.0       70+        Married       0   
            4796                 0.0       70+        Married       0   
            4953                 0.0       70+        Married       0   
            4953                 0.0       70+        Married       0   

                    family_size no_of_children  income_bracket  brand  \
customer_id item_id                                                     
1           4315              2            NaN               4   2902   
            4577              2            NaN               4    115   
            4796              2            NaN               4    278   
            4953              2            NaN               4   1061   
            4953              2            NaN               4   1061   

                      brand_type        category  day  month  year  week  \
customer_id item_id                                                        
1           4315     Established  Pharmaceutical   20      6  2013    25   
            4577     Established         Grocery    7     11  2012    45   
            4796     Established         Grocery   18     10  2012    42   
            4953     Established         Grocery   21      2  2012     8   
            4953     Established         Grocery   17      5  2012    20   

                     weekday  
customer_id item_id           
1           4315           3  
            4577           2  
            4796           3  
            4953           1  
            4953           3

In [21]:
mymap1 = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
mymap2 = {1:'January', 2:'February', 3:'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}

transactions_by_customer['weekday'] = transactions_by_customer['weekday'].map(mymap1)
transactions_by_customer['month'] = transactions_by_customer['month'].map(mymap2)

pd.set_option('display.max_columns', None)
transactions_by_customer.head()

date  quantity  selling_price  other_discount  \
customer_id item_id                                                       
1           4315    2013-06-20         1         201.97             0.0   
            4577    2012-11-07         2         120.40             0.0   
            4796    2012-10-18         1         106.50             0.0   
            4953    2012-02-21         1         142.12             0.0   
            4953    2012-05-17         1         142.12             0.0   

                     coupon_discount age_range marital_status  rented  \
customer_id item_id                                                     
1           4315                 0.0       70+        Married       0   
            4577                 0.0       70+        Married       0   
            4796                 0.0       70+        Married       0   
            4953                 0.0       70+        Married       0   
            4953                 0.0       70+        Married       0   

                    family_size no_of_children  income_bracket  brand  \
customer_id item_id                                                     
1           4315              2            NaN               4   2902   
            4577              2            NaN               4    115   
            4796              2            NaN               4    278   
            4953              2            NaN               4   1061   
            4953              2            NaN               4   1061   

                      brand_type        category  day     month  year  week  \
customer_id item_id                                                           
1           4315     Established  Pharmaceutical   20      June  2013    25   
            4577     Established         Grocery    7  November  2012    45   
            4796     Established         Grocery   18   October  2012    42   
            4953     Established         Grocery   21  February  2012     8   
            4953     Established         Grocery   17       May  2012    20   

                       weekday  
customer_id item_id             
1           4315      Thursday  
            4577     Wednesday  
            4796      Thursday  
            4953       Tuesday  
            4953      Thursday

In [22]:
transactions_by_customer.to_csv('transactions_by_customer_DW.csv')